In [39]:
import numpy as np
import galois
import itertools
from scipy import sparse
GF = galois.GF(2)

In [3]:
# https://scialert.net/fulltext/?doi=tasr.2012.929.934
def girth(H, g):
    # girth can only be even in bipartite graphs
    combs = list(itertools.combinations(np.arange(H.shape[0]), g//2))
    for c in combs:
        w = np.zeros(H.shape[1])
        for r in range(g//2):
            w += np.array(H[c[r]])
        if (np.count_nonzero(w == 2) == g//2):
            return True
    return False

In [543]:
# configuration model
# https://en.wikipedia.org/wiki/Configuration_model

n = 40000
deg_c = 10 # w_r. Every check has this many bits in it, d
deg_v = 5 # w_c. Every bit is in this many checks, c
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), np.int64)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            # if (girth(H, 4)):
            #     H[cs[c_ind]][vs[v_ind]] = 0
            #     break
            vs = np.delete(vs, v_ind)
            cs = np.delete(cs, c_ind)

H = sparse.csc_matrix(H)
# H = GF(H)

In [462]:
H = np.loadtxt('./good_ldpc_codes/16_4_3.txt', dtype=np.int8)
H = sparse.csc_matrix(H)

In [ ]:
while (girth(H, 4)):
    arange = np.arange(H.shape[0])
    i = np.random.choice(arange, replace=False)
    j = np.random.choice(arange, replace=False)
    bits_i = np.where(H[i])[0]
    bits_j = np.where(H[j])[0]
    i_ind = np.random.choice(bits_i)
    j_ind = np.random.choice(bits_j)
    
    if (i_ind == j_ind or i_ind in bits_j or j_ind in bits_i): 
        continue
    
    v = GF(np.zeros(H.shape[1], dtype=np.int8))
    v[i_ind] = v[j_ind] = 1
    H[i] += v
    H[j] += v


In [ ]:
def rref(H):
    r = 0
    for c in range(k, n):
        if (H[r, c] == 0):
            # need to swap
            for r2 in range(r+1, num_checks):
                if (H[r2, c] == 1):
                    temp = H[r, :].copy()
                    H[r, :] = H[r2, :]
                    H[r2, :] = temp
                    break
                    
        if (H[r, c] == 0):
            print('H is singular')

        # cancel out other rows
        for r2 in range(r+1, num_checks):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        # # back substitution
        for r2 in range(0, r):
            if (H[r2, c] == 1):
                H[r2, :] = H[r2, :] + H[r, :]

        r += 1
    return H

rref_H = GF(rref(GF(H.copy())))

G = GF(np.hstack((GF(np.eye(k, dtype=np.int64)), rref_H[:,0:k].T)))
print(np.any(H @ G.T))

In [36]:
for i in range(2**k):
    m = [int(j) for j in bin(i)[2:].zfill(k)]
    c = GF(m) @ G
    print(np.array(c), np.count_nonzero(c))

[0 0 0 0 0 0 0 0] 0
[0 0 0 1 0 0 1 0] 2
[0 0 1 0 0 0 1 0] 2
[0 0 1 1 0 0 0 0] 2
[0 1 0 0 0 0 1 1] 3
[0 1 0 1 0 0 0 1] 3
[0 1 1 0 0 0 0 1] 3
[0 1 1 1 0 0 1 1] 5
[1 0 0 0 1 1 0 0] 3
[1 0 0 1 1 1 1 0] 5
[1 0 1 0 1 1 1 0] 5
[1 0 1 1 1 1 0 0] 5
[1 1 0 0 1 1 1 1] 6
[1 1 0 1 1 1 0 1] 6
[1 1 1 0 1 1 0 1] 6
[1 1 1 1 1 1 1 1] 8


In [544]:
def flip_alt3(y, H):
    def determine_flippable(w):
        flippable = []
        for v in range(H.shape[1]):
            checks = np.nonzero(H[:, v])[0]
            unsatisfied = np.sum([x(j, w) for j in checks])
            deg = len(checks)
            if (2 * unsatisfied >= deg):
                flippable.append(v)
        return flippable

    # evaluates the lth parity check with recieved vector y
    x = lambda l, y: np.sum([y[k] for k in np.where(H[l])]) % 2 

    w = y.copy()
    flippable = determine_flippable(w)
    while flippable:
        z = GF(np.zeros(H.shape[1], dtype=np.int8))
        z[flippable[0]] = 1
        w += z
        flippable = determine_flippable(w)

    if np.any(H @ w):
        return "FAIL"
    else:
        # where w is the deduced word
        return w

In [545]:
indices = [set(H[:,i].indices) for i in range(H.shape[1])]

In [553]:
def flip_alt(e, H, B):
    # e is the error
    # H is the parity check matrix
    # B is the number by which the syndrome has to decrease each iteration 
    sigma = set(np.nonzero(H.dot(e) % 2)[0])
    e = set()

    while sigma:
        valid = False
        for i in range(H.shape[1]):
            # loop through all variables
            neighborhood = indices[i]
            new_sigma = sigma ^ neighborhood

            if len(new_sigma) <= len(sigma) - B:
                e = e ^ set([i])
                sigma = new_sigma
                valid = True
                break

        if (not valid):
            break
    return e

In [547]:
def flip_alt2(y, H):
    w = y.copy()
    syndrome = H.dot(w) % 2

    while True:
        terminate = True
        for v in range(H.shape[1]):
            checks = indices[v]
            unsatisfied = syndrome[checks]
            if (2 * np.sum(unsatisfied) > deg_v):
                if (w[v]):
                    np.put(w, v, [0])
                else:
                    np.put(w, v, [1])

                np.put(syndrome, checks[unsatisfied == 0], [1])
                np.put(syndrome, checks[unsatisfied == 1], [0])
                terminate = True
                break
        if (not terminate):
            continue
        
        if np.any(H @ w):
            return "FAIL"
        else:
            return w

In [557]:
p = 0.01
num_errors = 100
sum = 0
for i in range(1):
    e = np.array([1] * num_errors + [0] * (n - num_errors))
    np.random.shuffle(e)
    e_hat = flip_alt(e, H, 5)

    if (set(np.where(e)[0]) == e_hat):
        sum += 1

500
495
490
485
480
475
470
465
460
455
450
445
440
435
430
425
420
415
410
405
400
395
390
385
380
375
370
365
360
355
350
345
340
335
330
325
320
315
310
305
300
295
290
285
280
275
270
265
260
255
250
245
240
235
230
225
220
215
210
205
200
195
190
185
180
175
170
165
160
155
150
145
140
135
130
125
120
115
110
105
100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10
5


In [558]:
sum

1

In [559]:
len(e_hat ^ set(np.where(e)[0]))

0